# Continuous Control

---




# DDPG Agent 

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import deque
from statistics import mean, stdev

from ENV import environment_loader

In [2]:
%load_ext autoreload
%autoreload 2
from AGENT import DDPG_Agent


--- General Training parameters ---
DEVICE:  cpu

--- Replay Buffer parameters ---
BUFFER_SIZE:  100000
UPDATE_EVERY:  4
BATCH_SIZE:  100

--- Learning parameters ---
GAMMA:  0.99
LR_ACTOR:  0.0001
LR_CRITIC:  0.0001
CRITERION:  <function mse_loss at 0x000001D8FD564EA0>
WEIGHT_DECAY:  0
TAU:  0.001
C:\Users\danie\anaconda3\envs\drlnd\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
env, state_size, action_size, brain_name, num_agents = environment_loader("Reacher_Single/Reacher.exe", no_graphics = True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00

In [4]:
def DDPG_eval(agent, env, max_t=100):
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    episode_length = 0                                     # measure number of steps allowed before done flag
    for i in range(max_t):
        # actions = np.random.randn(num_agents, action_size) # select an action (for each agent); randomly!
        actions =  np.random.randn(num_agents, action_size) if agent is None else agent.act(states[0]) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        episode_length +=1
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Episode length is {}'.format(episode_length))
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [5]:
DDPG_eval(None, env, max_t=1)

Episode length is 1
Total score (averaged over agents) this episode: 0.0


In [6]:
def DDPG_train(agent, env, max_t=100, num_episodes = 1, print_every=100, existing_score=[]):
    score_history = existing_score
    print("Initial Score History: ", score_history)
    scores_deque = deque(maxlen=print_every)
    last_running_mean = float('-inf')

    for episode in range(num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        agent.reset()                                          # reset the noise in the agent
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        score = 0

        for i in range(max_t):
            actions = agent.act(states[0])                     # select an action (for each agent)
            #actions = np.clip(actions, -1, 1)                 # all actions between -1 and 1. This is already done in the model
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            #print("s {}, a {}, r {}, n_s {}, d{}".format(states[0], actions, rewards[0], next_states[0], dones[0]))
            agent.step(states[0], actions, rewards[0], next_states[0], dones[0])

            #scores += env_info.rewards                         # update the score (for each agent)
            score += env_info.rewards[0]

            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        

        #returns_in_episode = np.mean(scores)
        returns_in_episode = score
        scores_deque.append(returns_in_episode)
        score_history.append(returns_in_episode)
        if episode > print_every:
            if mean(scores_deque) > last_running_mean:
                    print("")
                    print('Last {} was better, going to save it'.format(print_every))
                    torch.save(new_agent.actor_local.state_dict(), 'checkpoint_actor.pth')
                    torch.save(new_agent.critic_local.state_dict(), 'checkpoint_critic.pth')
                    last_running_mean = mean(scores_deque)

        print("\r", 'Total score (averaged over agents) {} episode: {} | \tAvarage in last {} is {}'.format(episode, returns_in_episode, print_every, np.mean(scores_deque)), end="")
        ##print("\r", 'Scores in {} episode: {}'.format(episode, scores), end="")
    

    return score_history

In [7]:
import os.path

filename="checkpoint_actor.pth"
path = ""

def fileAtLocation(filename, path):
    return os.path.exists(path + filename)

def load_previous(new_agent):
    loaded_agent = new_agent
    if fileAtLocation(filename, path):
        print("Found previous trained Agent, going to load them!")
        loaded_agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
        loaded_agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))
    return loaded_agent

In [8]:
new_agent = DDPG_Agent(state_size, action_size, 1, actor_hidden = [256, 256], critic_hidden=[256, 256])
new_agent = load_previous(new_agent)


--- Agent Params ---
Going to train on cpu
Learning Rate:: Actor: 0.0001 | Critic: 0.0001
Replay Buffer:: Buffer Size: 100000 | Sampled Batch size: 100

Actor paramaters:: Input: 33 | Hidden Layers: [256, 256] | Output: 4
Critic paramaters:: Input: 33 | Hidden Layers: [260, 256] | Output: 1
Actor(
  (fc_in): Linear(in_features=33, out_features=256, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=256, out_features=256, bias=True)
  )
  (fc_out): Linear(in_features=256, out_features=4, bias=True)
)
Critic(
  (fc_in): Linear(in_features=33, out_features=256, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=260, out_features=256, bias=True)
  )
  (fc_out): Linear(in_features=256, out_features=1, bias=True)
)


Found previous trained Agent, going to load them!


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
##env.no_graphics = True
score_history = DDPG_train(new_agent, env, max_t = 1001, num_episodes = 150)
#print(score_history)

In [ ]:
import csv

with open('last_score.csv', newline='') as f:
    reader = csv.reader(f)
    read_score_history = list(reader)[0]

In [ ]:
new_score_history = DDPG_train(new_agent, env, max_t = 1001, num_episodes = 150, existing_score=read_score_history)

In [ ]:
with open("last_score.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(score_history)

In [ ]:
print(score_history)
print(len(score_history))
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(score_history)+1), score_history)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2
from AGENT import DDPG_Agent

new_agent = DDPG_Agent(state_size, action_size, 5)
new_agent = load_previous(new_agent)

In [ ]:
DDPG_eval(new_agent, env, max_t = 100000000000)

In [ ]:
env.close()

In [ ]:
torch.cuda.is_available()